# Seq2seq와 Attention

### 목표
1. 언어 모델이 발전해 온 과정 알기
2. 번역에서 기존 RNN 기법의 한계를 알고, 이를 개선한 seq2seq 이해하기
3. seq2seq을 발전시킨 attention 알기

<hr>

## 1. 언어 모델

- 언어 모델이란 주어진 단어들을 보고 다음 단어를 맞추는 모델이다. 즉 단어의 시퀀스를 보고 다음 단어에 확률을 할당하는 모델이다.
- 다른 말로, 단어 시퀀스에 대한 확률 분포라고 할 수 있다. 이 확률 분포를 가지고 단어의 시퀀스를 뽑을 수도 있다. 즉 언어 모델로 텍스트를 생성해낼 수 있다는 뜻이다.
- 수학적으로 표현하자면, 다음과 같다.
![image](https://user-images.githubusercontent.com/80008411/136878437-a638efd5-38cf-495d-bbbf-f887d1a61c96.png)

### 1) 통계적 언어 모델(Stattistical Language Model)

- 딥러닝 등장 전 대표적인 모델이었다.

#### - 유니그램 언어 모델

- 가장 단순한 언어 모델
- 각 단어가 서로 독립이라고 가정한다. 즉 n개 단어가 동시에 나타날 확률은 다음과 같이 나타낼 수 있다.
![image](https://user-images.githubusercontent.com/80008411/136878991-55e422bb-73d2-4fe8-903f-476040fd0a2d.png)

- 단어의 시퀀스를 고려한다기보다는 단어 셋을 상정한다. 단어 시퀀스의 등장 확률이 각 단어 발생 확률의 곱으로 정의돼 있기 때문이다.

#### - 한계
- 언어모델 $\theta$를 학습한다는 것은 각 단어의 등장 확률을 추정한다는 뜻이다. 그런데 주어진 데이터가 모집단 전체를 포괄하는 게 아니라 일부이기 때문에, 학습 corpus에 존재하지 않는 단어일 경우 확률이 0이 된다. 즉 범용적인 모델을 구축하기 어렵다.
- 또 조사, 어미 등 기능적 단어가 주제 단어보다 빈도가 훨씬 높아서 원하는 결과를 내기 어렵다.
- 이를 극복하기 위한 방법으로 평탄화(smoothing) 등 기법이 제안되기도 한다.
- n-gram 언어 모델은 바로 앞의 n-1개의 단어만 참고한다. 때문에 충분한 데이터를 관측하지 못하면 언어를 정확히 모델링하지 못하는 희소 문제(sparsity problem)이 있다.

### 2) 신경망 언어 모델 NNLM

- 통계적 언어 모델의 단점을 개선
- 시초는 Feed-Forward 모델인데, 지금의 Embedding 레이어의 아이디어 모델이다.
- 언어의 모델이 단어의 유사도를 학습할 수 있도록 해서, 훈련 corpus에 없는 단어 시퀀스에 대한 예측이라도 유사한 단어가 사용된 시퀀스를 참고할 수 있다는 아이디어로 탄생했다. 즉 단어 간 유사도를 반영한 벡터를 만드는 워드 임베딩을 이용하는 것이다.

#### - 피드 포워드 NNML

- 예문 : "what will the fat cat sit on"
- 우선 훈련 코퍼스가 준비된 상태에서 모든 단어를 숫자로 원핫 인코딩한다. 단어 셋의 크기를 차원으로 가지는 이 원핫 벡터가 신경망의 입력이면서 예측을 위한 레이블이 된다.
- 아래처럼 4개 층으로 이루어진 신경망을 보자. 윈도우 크기는 4로 정했다.

![image](https://user-images.githubusercontent.com/80008411/136884291-741789ab-0b50-4657-97e7-9ab15bc82322.png)

- 투사층(projection layer)이 일반 은닉층과 구별되는 특징은 가중치 행렬과의 연산은 이루어지지만 활성화 함수가 존재하지 않는다는 점이다.
- 예를 들어 투사층의 크기를 M, 단어 셋의 크기를 V라고 하면, 각 입력 단어들은 V x M 크기의 가중치 행렬과 곱해진다. 이는 가중치 행렬의 i번째 행을 그대로 읽어오는 것과 동일하며, lookup table이라고 불린다. 이 단어 벡터를 임베딩 벡터라고 한다.

![image](https://user-images.githubusercontent.com/80008411/136885054-9f4cc475-42b2-416d-8c44-95b17a8a8677.png)

- 즉 각 단어가 테이블 룩업을 통해 임베딩 벡터로 변경되고, 투사층에서 모든 임베딩 벡터들의 값이 연결된다.

![image](https://user-images.githubusercontent.com/80008411/136885421-8fa9db16-b5b6-4e01-9862-bf096317e2c5.png)

- 투사층의 결과는 h의 크기를 가지는 은닉층을 지난다. 즉 가중치가 곱해지고, 편향이 더해져, 활성화 함수(tanh)의 입력이 된다.
- 은닉층의 출력은 이제 V의 크기를 가지는 출력층으로 간다. 또 다시 가중치와 곱해지고 편향이 더해지면, 최초 입력과 마찬가지로 V 차원의 벡터를 얻는다. 마지막으로 활성화 함수(softmax)를 지나면서 0과 1 사이 실수값을 가지게 된다.
- 이때 이 예측된 확률값이 실제 y와 가까워지도록 하기 위해 손실 함수(cross-entropy)를 사용한다. 그리고 역전파 과정에서 가중치 행렬과 임베딩 벡터값들이 학습된다.

![image](https://user-images.githubusercontent.com/80008411/136885591-a5b7cf86-e9c4-4bf0-aaf4-6635bb4698fe.png)

#### - 이점
- 단어의 유사도를 표현하기 위해 밀집 벡터를 사용하고, 이를 통해 희소 문제를 해결한다.
- 또 더 이상 모든 n-gram을 저장하지 않아도 되므로 저장 공간의 이점이 있다.

#### - 한계
- 그러나 n-gram과 마찬가지로 정해진 n개 단어만을 참고하므로, 버려지는 단어들과 문맥 정보가 생긴다.
- 또 학습 코퍼스의 문장 길이는 전부 다르기 때문에, 매번 다른 길이의 입력 시퀀스에 대해서도 모델이 처리할 수 있어야 한다.

<hr>

## 2. Seq2seq 알기

- 앞선 n개의 단어를 고려한 NNLM의 한계는 여러 단어의 워드 임베딩을 합쳐서 고정된 크기의 weight를 linear로 처리하는 방식은 유연하지 못하다는 것이다.
- 즉 단어의 개수에 무관하게 처리할 수 있는 네트워크가 필요했고, 이로부터 RNN이 등장한다.
- RNN은 고정된 weight가 선언되는 것은 동일하지만 입력을 순차적으로 적립하는 방식을 채택함으로써 유동적인 크기의 입력을 처리할 수 있다.

### 1) RNN의 한계
- 단어가 자체적으로 의미를 가질 수 있는 임베딩을 도입하고, 입력의 유연성을 위해 RNN도 적용했는데, 여전히 두 가지 한계가 존재했다.
- 1) 하나의 weight에 입력을 적립하다 보니 입력이 길어질수록 이전 입력에 대한 정보가 소실되는 기울기 소실 문제가 있다.
- 이는 LSTM을 통해 해결할 수 있게 되었다.

![image](https://user-images.githubusercontent.com/80008411/136898539-0299eaf6-e051-4916-973f-58f286ba8211.png)

- 2) 단어 단위로 입력과 출력을 순환하기 때문에 문장 생성에는 적합하지만 번역에 사용하기는 어렵다. 또 입력과 번역 문장의 길이가 같다는 보장도 없기 때문에 문장 전체를 보고 생성하는 구조가 필요했고, 이를 위해 구글은 Seq2seq을 제안한다.

![image](https://user-images.githubusercontent.com/80008411/136903612-ef9e38d0-42bf-41a2-882e-48757c6a0b49.png)

### 2) seq2seq 개념

- 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 다양한 분야에서 사용되는 모델.
- 대표적으로 챗본, 기계 번역 등. 즉 입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성하면 챗봇을 만들 수 있고, 입력과 출력 시퀀스를 각각 입력과 번역 문장으로 만들면 번역기를 만들 수 있다.
- 그 외에도 summarization, Speech to Text 등에서 쓰일 수 있다.

![image](https://user-images.githubusercontent.com/80008411/136904585-ce78a5bf-56a1-496c-b74a-98dc380b75fa.png)

- 구성은 크게 2개의 아키텍처, 즉 인코더와 디코더로 되어 있다.
- 인코더는 입력 문장의 모든 단어들을 순차적으로 입력 받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만드는데, 이를 컨텍스트 벡터라고 한다. 입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더가 이를 디코더로 전송한다.
- 디코더는 컨텍스트 벡터를 받아 번역된 단어를 한 개씩 순차적으로 출력한다.

![image](https://user-images.githubusercontent.com/80008411/136905642-d9cccd92-7321-4bd9-9fb7-2adf8aa6a7f4.png)

![image](https://user-images.githubusercontent.com/80008411/136921354-91249bc0-bb2e-4a48-b249-a929299db527.png)

### 3) 구현

In [2]:
# lstm encoder
import tensorflow as tf

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(enc_units) # return_sequences 매개변수를 기본값 False로 전달

    def call(self, x):
        print("입력 Shape:", x.shape)

        x = self.embedding(x)
        print("Embedding Layer를 거친 Shape:", x.shape)

        output = self.lstm(x)
        print("LSTM Layer의 Output Shape:", output.shape)

        return output

In [3]:
vocab_size = 30000
emb_size = 256
lstm_size = 512
batch_size = 1
sample_seq_len = 3

print("Vocab Size: {0}".format(vocab_size))
print("Embedidng Size: {0}".format(emb_size))
print("LSTM Size: {0}".format(lstm_size))
print("Batch Size: {0}".format(batch_size))
print("Sample Sequence Length: {0}\n".format(sample_seq_len))

Vocab Size: 30000
Embedidng Size: 256
LSTM Size: 512
Batch Size: 1
Sample Sequence Length: 3



In [4]:
encoder = Encoder(vocab_size, emb_size, lstm_size)
sample_input = tf.zeros((batch_size, sample_seq_len))

sample_output = encoder(sample_input)    # 컨텍스트 벡터로 사용할 인코더 LSTM의 최종 State값

입력 Shape: (1, 3)
Embedding Layer를 거친 Shape: (1, 3, 256)
LSTM Layer의 Output Shape: (1, 512)


In [8]:
# lstm decoder
# Encoder 구현에 사용된 변수들을 이어 사용함에 유의!
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(dec_units,
                                         return_sequences=True) # return_sequences 매개변수를 True로 설정
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.softmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, x, context_v):  # 디코더의 입력 x와 인코더의 컨텍스트 벡터를 인자로 받는다. 
        print("입력 Shape:", x.shape)

        x = self.embedding(x)
        print("Embedding Layer를 거친 Shape:", x.shape)

        context_v = tf.repeat(tf.expand_dims(context_v, axis=1),
                              repeats=x.shape[1], axis=1)
        x = tf.concat([x, context_v], axis=-1)  # 컨텍스트 벡터를 concat 해준다
        print("Context Vector가 더해진 Shape:", x.shape)

        x = self.lstm(x)
        print("LSTM Layer의 Output Shape:", x.shape)

        output = self.fc(x)
        print("Decoder 최종 Output Shape:", output.shape)

        return self.softmax(output)

In [6]:
print("Vocab Size: {0}".format(vocab_size))
print("Embedidng Size: {0}".format(emb_size))
print("LSTM Size: {0}".format(lstm_size))
print("Batch Size: {0}".format(batch_size))
print("Sample Sequence Length: {0}\n".format(sample_seq_len))

Vocab Size: 30000
Embedidng Size: 256
LSTM Size: 512
Batch Size: 1
Sample Sequence Length: 3



In [7]:
decoder = Decoder(vocab_size, emb_size, lstm_size)
sample_input = tf.zeros((batch_size, sample_seq_len))

dec_output = decoder(sample_input, sample_output)  # Decoder.call(x, context_v) 을 호출

입력 Shape: (1, 3)
Embedding Layer를 거친 Shape: (1, 3, 256)
Context Vector가 더해진 Shape: (1, 3, 768)
LSTM Layer의 Output Shape: (1, 3, 512)
Decoder 최종 Output Shape: (1, 3, 30000)


<hr>

## 3. Attention 알기

### 1) Bahdanau Attention

- seq2seq의 컨텍스트 벡터가 고정된 길이로 정보를 압축하는 것이 손실을 야기하는 문제가 제기되었다. 이를 해결하기 위해 기존 인코더의 최종 state 값만을 사용하는 방식이 아닌, 매 스텝의 hidden state를 활용해 컨텍스트 벡터를 구축하는 어텐션 메커니즘이 등장한다.

![image](https://user-images.githubusercontent.com/80008411/136922054-9c7331a8-bbf4-4e8b-816b-edf41de2944d.png)

#### - 구현

In [9]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W_decoder = tf.keras.layers.Dense(units)
        self.W_encoder = tf.keras.layers.Dense(units)
        self.W_combine = tf.keras.layers.Dense(1)

    def call(self, H_encoder, H_decoder):
        print("[ H_encoder ] Shape:", H_encoder.shape)

        H_encoder = self.W_encoder(H_encoder)
        print("[ W_encoder X H_encoder ] Shape:", H_encoder.shape)

        print("\n[ H_decoder ] Shape:", H_decoder.shape)
        H_decoder = tf.expand_dims(H_decoder, 1)
        H_decoder = self.W_decoder(H_decoder)

        print("[ W_decoder X H_decoder ] Shape:", H_decoder.shape)

        score = self.W_combine(tf.nn.tanh(H_decoder + H_encoder))
        print("[ Score_alignment ] Shape:", score.shape)

        attention_weights = tf.nn.softmax(score, axis=1)
        print("\n최종 Weight:\n", attention_weights.numpy())

        context_vector = attention_weights * H_decoder
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

W_size = 100

print("Hidden State를 {0}차원으로 Mapping\n".format(W_size))

attention = BahdanauAttention(W_size)

enc_state = tf.random.uniform((1, 10, 512))
dec_state = tf.random.uniform((1, 512))

_ = attention(enc_state, dec_state)

Hidden State를 100차원으로 Mapping

[ H_encoder ] Shape: (1, 10, 512)
[ W_encoder X H_encoder ] Shape: (1, 10, 100)

[ H_decoder ] Shape: (1, 512)
[ W_decoder X H_decoder ] Shape: (1, 1, 100)
[ Score_alignment ] Shape: (1, 10, 1)

최종 Weight:
 [[[0.06767835]
  [0.10397432]
  [0.08834428]
  [0.10051849]
  [0.07707576]
  [0.11265649]
  [0.15215074]
  [0.06919928]
  [0.11389282]
  [0.11450939]]]


### 2) Luong Attention

- Bahdanau의 방식을 약간 발전시킨 형태. 하나의 weight만을 사용해 score 함수를 계산한다.
- 어떤 벡터 공간에 매핑해주는 과정이 없으므로 weight의 크기는 단어 임베딩 크기와 동일해야 연산이 가능하다.
- 디코더의 현재 hidden state를 구하기 위해 한 스텝 이전의 hidden state를 활용하는 것은 연산적으로 비효율적이다. 

#### - 구현

In [11]:
class LuongAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(LuongAttention, self).__init__()
        self.W_combine = tf.keras.layers.Dense(units)

    def call(self, H_encoder, H_decoder):
        print("[ H_encoder ] Shape:", H_encoder.shape)

        WH = self.W_combine(H_encoder)
        print("[ W_encoder X H_encoder ] Shape:", WH.shape)

        H_decoder = tf.expand_dims(H_decoder, 1)
        alignment = tf.matmul(WH, tf.transpose(H_decoder, [0, 2, 1]))
        print("[ Score_alignment ] Shape:", alignment.shape)

        attention_weights = tf.nn.softmax(alignment, axis=1)
        print("\n최종 Weight:\n", attention_weights.numpy())

        attention_weights = tf.squeeze(attention_weights, axis=-1)
        context_vector = tf.matmul(attention_weights, H_encoder)

        return context_vector, attention_weights

emb_dim = 512

attention = LuongAttention(emb_dim)

enc_state = tf.random.uniform((1, 10, emb_dim))
dec_state = tf.random.uniform((1, emb_dim))

_ = attention(enc_state, dec_state)

[ H_encoder ] Shape: (1, 10, 512)
[ W_encoder X H_encoder ] Shape: (1, 10, 512)
[ Score_alignment ] Shape: (1, 10, 1)

최종 Weight:
 [[[0.01032101]
  [0.19615139]
  [0.2522048 ]
  [0.04303023]
  [0.43915242]
  [0.00119061]
  [0.048869  ]
  [0.0007583 ]
  [0.00088168]
  [0.00744056]]]


### 3) 미래 기법들

- [Google's Neural Machine Translation System](https://norman3.github.io/papers/docs/google_neural_machine_translation.html)
- 트랜스포머
    - Multi-Head Attention이라는 개념을 도입해 폭넓은 문맥을 파악하게 하고, 기존 RNN 구조를 완전히 탈피하여 연산 속도 측면에서도 혁신적으로 발전했다.

![image](https://user-images.githubusercontent.com/80008411/136923002-3747afd9-8cce-48fe-a6ae-5dd516558941.png)
